In [1]:
%cd /content/drive/MyDrive/TextToSpeech

/content/drive/MyDrive/TextToSpeech


In [2]:
!ls

config.json	     scale_stats.npy  tts_model.pkl
config_vocoder.json  TTS	      vocoder_model.pkl


In [3]:
# Library (Specific to Ubuntu)
# need it for char to phoneme conversion
! sudo apt-get install espeak

Reading package lists... Done
Building dependency tree       
Reading state information... Done
espeak is already the newest version (1.48.04+dfsg-5).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [4]:
%cd TTS
!pip install -r requirements.txt
!python setup.py install
!pip install tensorflow==2.2.0
%cd ..

/content/drive/MyDrive/TextToSpeech/TTS
     |████████████████████████████████| 245kB 5.7MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 32.4MB 86kB/s 
     |████████████████████████████████| 204kB 37.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for bokeh: filename=bokeh-1.4.0-cp37-none-any.whl size=23689207 sha256=bd9a60b01621d41261753dc1a703c0365fb91c8208921a5c0e2ab269f70b7038
  Stored in directory: /root/.cache/pip/wheels/fb/f8/47/09700d9a19cbcbf0b7a3130690b75c0d6ff80fbda0b1774c7c
Successfully built bokeh
ERROR: panel 0.11.3 has requirement bokeh<2.4.0,>=2.3.0, but you'll have bokeh 1.4.0 which is incompatible.
  Found existing installation: bokeh 2.3.3
    Uninstalling bokeh-2.3.3:
      Successfully uninstalled bokeh-2.3.3
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'build-lib' will not be supported in future versions. Please use the underscor

In [5]:
# TTS Function 
def tts(model, text, CONFIG, p):
    t_1 = time.time()
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs = synthesis(model, text, CONFIG, use_cuda, ap, speaker_id, style_wav=None,
                                                                             truncated=False, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars,
                                                                             backend='tf')
    waveform = vocoder_model.inference(torch.FloatTensor(mel_postnet_spec.T).unsqueeze(0))
    waveform = waveform.numpy()[0, 0]
    rtf = (time.time() - t_1) / (len(waveform) / ap.sample_rate)
    tps = (time.time() - t_1) / len(waveform)
    print(waveform.shape)
    print(" > Run-time: {}".format(time.time() - t_1))
    print(" > Real-time factor: {}".format(rtf))
    print(" > Time per step: {}".format(tps))
    IPython.display.display(IPython.display.Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    return alignment, mel_postnet_spec, stop_tokens,CONFIG.audio['sample_rate'], waveform

# Function To Convert Waveform to ".wav" file 
from scipy.io.wavfile import write
def createSoundFile(waveform,rate,nameOfFile='output.wav'):
  # scipy.io.wavfile.write(filename, rate, data)[source]¶
  write(nameOfFile, rate, waveform)  # Save as WAV file


# Loading Models

In [8]:
import os
import torch
import time
import IPython

from TTS.tf.utils.generic_utils import setup_model
from TTS.tf.utils.io import load_checkpoint
from TTS.utils.io import load_config
from TTS.utils.text.symbols import symbols, phonemes
from TTS.utils.audio import AudioProcessor
from TTS.utils.synthesis import synthesis

In [9]:
# runtime settings
use_cuda = False

In [10]:
# model paths
TTS_MODEL = "tts_model.pkl"
TTS_CONFIG = "config.json"
VOCODER_MODEL = "vocoder_model.pkl"
VOCODER_CONFIG = "config_vocoder.json"

In [11]:
# load configs
TTS_CONFIG = load_config(TTS_CONFIG)
VOCODER_CONFIG = load_config(VOCODER_CONFIG)

In [12]:
# load the audio processor
ap = AudioProcessor(**TTS_CONFIG.audio)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:./scale_stats.npy
 | > hop_length:256
 | > win_length:1024


In [13]:
# LOAD TTS MODEL
# multi speaker 
speaker_id = None
speakers = []

# load the model
num_chars = len(phonemes) if TTS_CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, len(speakers), TTS_CONFIG)
model.build_inference()
model = load_checkpoint(model, TTS_MODEL)
model.decoder.set_max_decoder_steps(1000)

 > Using model: Tacotron2
(1, None, 80)
(1, None, 80)


In [14]:
from TTS.vocoder.tf.utils.generic_utils import setup_generator
from TTS.vocoder.tf.utils.io import load_checkpoint

# LOAD VOCODER MODEL
vocoder_model = setup_generator(VOCODER_CONFIG)
vocoder_model.build_inference()
vocoder_model = load_checkpoint(vocoder_model, VOCODER_MODEL)
vocoder_model.inference_padding = 0

ap_vocoder = AudioProcessor(**VOCODER_CONFIG['audio'])    

 > Generator Model: multiband_melgan_generator
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:./scale_stats.npy
 | > hop_length:256
 | > win_length:1024


In [ ]:
# Saving the objects needed to create audio from text [NOT WORKING]
import pickle
def generatePickleFile(obj,pkl_filename):
  file_to_store = open(pkl_filename, "wb")
  pickle.dump(obj, file_to_store)
  file_to_store.close()

generatePickleFile(model,"model_object.pkl")
generatePickleFile(TTS_CONFIG,"TTS_CONFIG_object.pkl")
generatePickleFile(ap,"ap_object.pkl")

In [21]:
sentence =  "Bill got in the habit of asking himself “Is that thought true?” and if he wasn’t absolutely certain it was, he just let it go."
sentence =   "Hello I am chirag"
align, spec, stop_tokens,rate, wav = tts(model, sentence, TTS_CONFIG, ap)
createSoundFile(waveform=wav,rate=rate,nameOfFile='output.wav')

(34560,)
 > Run-time: 0.8991024494171143
 > Real-time factor: 0.5730637939025959
 > Time per step: 2.5989646436991516e-05
